In [2]:
import os
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as lp

In [3]:
import analyze2p.receptive_fields.utils as rfutils
import analyze2p.objects.sim_utils as su

import analyze2p.plotting as pplot
import analyze2p.aggregate_datasets as aggr
import analyze2p.utils as hutils


In [4]:
#### Plotting params
visual_areas, area_colors = pplot.set_threecolor_palette()
pplot.set_plot_params(labelsize=6, lw_axes=0.25)
bw_colors = dict((v, [0.7]*3) for v in visual_areas)

In [7]:
#### Set trace ID and FOV/state type
aggregate_dir = '/n/coxfs01/julianarhee/aggregate-visual-areas'
rootdir='/n/coxfs01/2p-data'
fov_type = 'zoom2p0x'
state = 'awake'
# ------------------------------------------------------------------
# Load all metdata and assigned cells
sdata, cells0 = aggr.get_aggregate_info(visual_areas=visual_areas, return_cells=True)

experiment_list = ['rfs10', 'rfs'] 
meta = sdata[sdata.experiment.isin(experiment_list)].copy()

/n/coxfs01/julianarhee/aggregate-visual-areas/dataset_info_assigned.pkl
Segmentation: missing 3 dsets


In [ ]:
# traceid = 'traces001'

# experiment = 'blobs'
# response_type='dff'
# responsive_test='nstds'
# if responsive_test=='nstds':
#     responsive_thr=7.0 if experiment=='rfs' else 10.
# else:
#     responsive_thr=0.05 if responsive_test=='ROC' else responsive_thr
# resp_desc = '%s_responsive-%s-thr%.2f' \
#                 % (response_type, responsive_test, responsive_thr)
# data_id = '|'.join([traceid, resp_desc])
# print(data_id)
# trial_epoch='plushalf' #'stimulus'

# CELLS = pd.concat([g for (va, dk), g in cells0.groupby(['visual_area', 'datakey'])\
#                 if not meta.query('@va == visual_area and @dk == datakey').empty])
# # Neural data
# NDATA0 = aggr.load_responsive_neuraldata(experiment, traceid=traceid,
#                       response_type=response_type, trial_epoch=trial_epoch,
#                       responsive_test=responsive_test, 
#                       responsive_thr=responsive_thr)
# if experiment not in ['rfs', 'rfs10']: # assigned in load_responsive_neuraldata()
#     NDATA0['experiment'] = experiment 

traces001|dff_responsive-nstds-thr10.00
/n/coxfs01/julianarhee/aggregate-visual-areas/dataset_info_assigned.pkl
Segmentation: missing 3 dsets
...loading: /n/coxfs01/julianarhee/aggregate-visual-areas/data-stats/aggr_blobs_trialmeans_traces001_nstds-thr-10.00_dff_plushalf.pkl
*Warning* <20190314_JC070_fov1> More than 1 pos? x: [-15.0 0.0], y: [-5.0 0.0]


In [ ]:
dst_dir = os.path.join(aggregate_dir, 'decoding', 'py3_by_ncells')

In [8]:
# Load all metdata and assigned cells
sdata, cells0 = aggr.get_aggregate_info(visual_areas=visual_areas, return_cells=True)
experiment_list = ['rfs10', 'rfs']
rf_meta = sdata[sdata.experiment.isin(experiment_list)].copy()

/n/coxfs01/julianarhee/aggregate-visual-areas/dataset_info_assigned.pkl
Segmentation: missing 3 dsets


In [16]:
import analyze2p.decoding as dec

In [10]:
do_spherical_correction=False
response_type='dff'
# CELLS = pd.concat([g for (va, dk), g in cells0.groupby(['visual_area', 'datakey'])\
#                 if not meta.query('@va == visual_area and @dk == datakey').empty])

rf_fit_desc = rfutils.get_fit_desc(response_type=response_type, 
                                 do_spherical_correction=do_spherical_correction)
rfdata = rfutils.aggregate_rfdata(rf_meta, cells0, fit_desc=rf_fit_desc,
                                 reliable_only=False)

N dpaths: 62, N unfit: 0
N datasets included: 62, N sessions excluded: 4


In [12]:
rfdata[['visual_area', 'datakey', 'experiment']].drop_duplicates()\
.groupby(['visual_area', 'datakey']).count().max()

experiment    2
dtype: int64

In [16]:
importlib.reload(rfutils)

<module 'analyze2p.receptive_fields.utils' from '/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/rat-2p-area-characterizations/analyze2p/receptive_fields/utils.py'>

In [17]:
rfdf = rfutils.average_rfs(rfdata, keep_experiment=False)


In [18]:
rfdf[['visual_area', 'datakey', 'experiment']].drop_duplicates()\
    .groupby(['visual_area', 'datakey']).count().max()

experiment    1
dtype: int64

In [20]:
dk = '20190509_JC078_fov1'
va = 'V1'
experiment='blobs'

In [22]:
curr_rfs = rfdf[(rfdf.visual_area==va) & (rfdf.datakey==dk)].copy()
curr_rfs.shape

(5, 30)

In [86]:
importlib.reload(su)


<module 'analyze2p.objects.sim_utils' from '/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/rat-2p-area-characterizations/analyze2p/objects/sim_utils.py'>

In [39]:
dst_dir = os.path.join(aggregate_dir, 'receptive-fields', 'dataframes')
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)
print(dst_dir)

poly_fpath = os.path.join(dst_dir, 'average_polys.pkl')
if os.path.exists(poly_fpath):
    with open(poly_fpath, 'rb') as f:
        POLYS = pkl.load(f)
else:
    POLYS = None

/n/coxfs01/julianarhee/aggregate-visual-areas/receptive-fields/dataframes


In [35]:
# Process all cells in fov
cols = [c for c in rfdf.columns if c!='visual_area']
by_dkey = rfdf[cols].drop_duplicates()
# Add new, if needed
check_these=[]
add_list=[]
for dk, curr_rfs in by_dkey.groupby('datakey'):
    if POLYS is not None:
        found_ = POLYS[(POLYS.datakey==dk)]
        need_ = curr_rfs[curr_rfs['cell'].isin(found_['cell'].values)]
    else:
        need_ = curr_rfs.copy()
    rf_polys, check_rfs = su.get_rf_polys(need_, check_invalid=True)
    if len(check_rfs)>0:
        check_these[dk]= check_rfs
    rf_polys['datakey'] = dk
    add_list.append(rf_polys)

#     rfs_ = params_deg_to_pixels(curr_rfs)
#     rf_screen = rf_to_screen(ri, rfs_, resolution=resolution)
#     rpoly = image_to_poly(rf_screen.astype(np.uint8))
    

KeyboardInterrupt: 

In [29]:
rfs_ = su.params_deg_to_pixels(curr_rfs)
for ri, r_ in rfs_.groupby('cell'):
    rf_screen = su.rf_to_screen(ri, r_)
    rpoly = su.image_to_poly(rf_screen.astype(np.uint8))

ValueError: invalid literal for int() with base 10: 'x0_pix'

In [38]:
rf_polys

,poly,cell
0,"POLYGON ((1165 748, 1164 749, 1163 749, 1162 7...",177.0
1,"POLYGON ((981 702, 980 703, 969 703, 968 704, ...",180.0
2,"POLYGON ((859 547, 858 548, 846 548, 845 549, ...",191.0
3,"POLYGON ((1086 635, 1085 636, 1081 636, 1080 6...",206.0
4,"POLYGON ((841 588, 840 589, 835 589, 834 590, ...",209.0
5,"POLYGON ((1058 529, 1057 530, 1051 530, 1050 5...",271.0
6,"POLYGON ((916 471, 915 472, 906 472, 905 473, ...",272.0
7,"POLYGON ((1168 616, 1167 617, 1164 617, 1163 6...",289.0
8,"POLYGON ((1711 568, 1710 569, 1709 569, 1708 5...",309.0
9,"POLYGON ((1556 459, 1555 460, 1550 460, 1549 4...",317.0


In [32]:
r_

,datakey,animalid,session,cell,x0,y0,fwhm_x,fwhm_y,theta,offset,...,minor_axis,anisotropy,theta_Mm_c,aniso_index,visual_area,experiment,x0_pix,y0_pix,fwhm_x_pix,fwhm_y_pix
197.0,20190508_JC083_fov1,JC083,20190508,197.0,53.957145,2.548797,27.746813,13.209007,0.060078,-0.005116,...,5.620854,0.354963,0.060078,-0.312338,Lm,average_rfs,1827,581,445.336347,212.004559
197.0,20190508_JC083_fov1,JC083,20190508,197.0,53.957145,2.548797,27.746813,13.209007,0.060078,-0.005116,...,5.620854,0.354963,0.060078,-0.312338,V1,average_rfs,1827,581,445.336347,212.004559


In [92]:
stim_polys = su.get_stimulus_polys(dk, experiment) 

In [93]:
o_=[]
for ri, rf_poly in rf_polys[['cell', 'poly']].values:
    res = su.cell_overlap_with_stimuli(ri, rf_poly, stim_polys)
    o_.append(res)
           

In [32]:
for si, spoly in enumerate(stim_polys['poly'].values):
    si

In [38]:
rfutils.calculate_overlap(rf_poly, spoly.buffer[0])

TypeError: 'method' object is not subscriptable

In [44]:
rf_poly + spoly

TypeError: unsupported operand type(s) for +: 'Polygon' and 'Polygon'

In [45]:
 rf_poly.intersection(spoly)

TopologyException: Input geom 0 is invalid: Self-intersection at or near point 1917 543 at 1917 543


TopologicalError: The operation 'GEOSIntersection_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x2b6bf9a23ed0>

In [52]:
from shapely.validation import make_valid

ImportError: cannot import name 'make_valid' from 'shapely.validation' (/n/coxfs01/2p-pipeline/envs/rat2p/lib/python3.7/site-packages/shapely/validation.py)

In [51]:
validation.make_valid


AttributeError: module 'shapely.validation' has no attribute 'make_valid'

In [ ]:
rf_lim = 'percentile'
# cells_lim = dec.get_cells_with_matched_rfs(cells0, sdata, rf_lim='percentile')
rfdf = dec.get_rfdf(cells0, sdata, do_spherical_correction=False)
cells_RF = dec.get_cells_with_rfs(cells0, rfdf)
cells_lim, sz_limits = dec.limit_cells_by_rf(cells_RF, rf_lim=rf_lim)

sz_lower, sz_upper = sz_limits.loc['lower'].max(), lims.loc['upper'].min()
sz_limits

In [ ]:
print(cells_RF[['visual_area', 'datakey', 'cell']].drop_duplicates()\
      ['visual_area'].value_counts().to_markdown())

In [ ]:
%matplotlib notebook
fig, ax = pl.subplots(figsize=(5,4))
sns.stripplot(x='visual_area', y='std_avg', hue='visual_area', 
             data=cells_RF, ax=ax, order=visual_areas, 
             palette=area_colors, s=2, zorder=0)
sns.pointplot(x='visual_area', y='std_avg',
             data=cells_RF, ax=ax, order=visual_areas, join=False, ci='sd',
             estimator=np.median, color='k', markers='_', scale=1, zorder=1000,
             errwidth=1)

# sz_upper_lim=16.6
# sz_lower_lim=6.9
ax.axhline(y=sz_lower, color='k', ls=':', lw=0.5)
ax.axhline(y=sz_upper, color='k', ls=':', lw=0.5)
ax.legend_.remove()

fig.text(0.01, 0.95, 'Distribution of avg RF size, with median (%s)' % rf_lim)

pl.savefig(os.path.join(dst_dir, 'RF_sizes_%s.svg' % rf_lim))


In [ ]:
# Compare overlaps

In [ ]:
m1 = 'M%i' % 0
m2 = 'M%i' % 106
#images, impaths = su.load_stimuli(root='/n/home00/juliana.rhee', return_paths=True)
#print("im shape:", images['M0'].shape)
images = su.load_stimuli()
f, axn = pl.subplots(1,2, figsize=(5,2))
axn[0].imshow(images[m1]); axn[0].set_title(m1);
axn[1].imshow(images[m2]); axn[1].set_title(m2);


In [ ]:
## Example fov
dk = '20190602_JC091_fov1'
va='Li'
experiment='blobs'

In [ ]:
rid=15
rf_screen = su.rf_to_screen(rid, curr_rfs, resolution=[1920, 1080])
%matplotlib notebook
pl.figure()
pl.imshow(rf_screen)

In [ ]:
stim_polys, stim_screen = su.get_stimulus_polys

In [ ]:
rf = su.image_to_poly(rf_screen)
blob = su.image_to_poly(stim_screen.astype(np.uint8))
odf = rfutils.calculate_overlap(rf, blob, r1='rf', r2='blob')